In [4]:
import numpy as np
import pandas as pd
import regex
import re
from PIL import Image
import torch
from torchvision.transforms import v2
from tqdm import tqdm
import os

In [5]:
# config에서 data 폴더 경로 가져오기
data_dir = "/data/ephemeral/home/data"  # 예: "config/data"

# users.csv 경로 만들기
user_data_path = os.path.join(data_dir, "users.csv")
user_df = pd.read_csv(user_data_path)

book_data_path = os.path.join(data_dir, "books.csv")
book_df = pd.read_csv(book_data_path)

train_rating_data_path = os.path.join(data_dir, "train_ratings.csv")
train_rating_df = pd.read_csv(train_rating_data_path)

## 

In [ ]:
import pandas as pd
import numpy as np

# 이전 질문에서 정의된 문자열을 리스트로 변환하는 함수
def str2list(x: str) -> list:
    '''문자열을 리스트로 변환하는 함수'''
    # 양쪽 대괄호 제거 및 ', ' 기준으로 분할
    return x[1:-1].split(', ')

# --- 고유 카테고리 추출 코드 ---

# 1. 'category' 열의 결측치(NaN)가 아닌 값만 필터링합니다.
#    apply 메서드가 NaN 값에 대해 오류를 일으키지 않도록 방지합니다.
valid_categories = book_df['category'].dropna()

# 2. 모든 카테고리 문자열을 리스트로 변환하고 하나의 리스트로 합칩니다.
#    lambda x: str2list(x)는 각 문자열을 리스트로 변환하고,
#    sum(..., [])는 그 모든 리스트를 하나의 큰 리스트로 결합합니다.
# all_categories = valid_categories.apply(lambda x: str2list(x)).sum(start=[])
all_categories = sum(valid_categories.apply(lambda x: str2list(x)), start=[])
# 3. 합쳐진 리스트에서 고유한(distinct) 카테고리만 추출합니다.
distinct_categories = sorted(list(set(all_categories)))

# 4. 결과를 출력합니다.
print("--- book_df의 고유 카테고리 목록 (Distinct Categories) ---")
for category in distinct_categories:
    print(category)


print("--- str2list 역할 파악 ---")
# 이 코드 결과가 없음을 보면 -> str2list 는 그저 문자열을 리스트로 변환하기만 한다.
for category in distinct_categories:
    if len(str2list(category)) >= 2:
        print(str2list(category))

# -----------------------------------------------------------
# 5. 최종 요구사항 분석 및 출력
# -----------------------------------------------------------

# 5-1. 카테고리 빈도 분석 및 TOP 100 출력
category_counts = Counter(all_categories)
df_counts = pd.DataFrame(category_counts.items(), columns=['Category', 'Count'])
df_counts = df_counts.sort_values(by='Count', ascending=False).reset_index(drop=True)

total_category_instances = len(all_categories)
df_counts['Percentage'] = (df_counts['Count'] / total_category_instances) * 100

print(f"총 카테고리 인스턴스 개수 (중복 포함): {total_category_instances}개")
print("\n[카테고리 등장 빈도 TOP 100 (상위 항목)]")
# Top 100 요청했으나, 데이터가 작으므로 전체 또는 상위 10개만 출력합니다.
top_n = min(100, len(df_counts))
print(df_counts.head(top_n).to_string(index=False, float_format="%.2f"))
print("-" * 50)


# 5-2. 분포 설명 및 이상치 카테고리 출력
# 이상치 정의: 전체 카테고리 중 단 1회만 등장한 카테고리
outlier_categories = df_counts[df_counts['Count'] == 1]

print(f"\n[카테고리 분포 분석]")
print(f"가장 많이 등장한 카테고리: {df_counts.iloc[0]['Category']} ({df_counts.iloc[0]['Count']}회)")
print(f"가장 적게 등장한 카테고리 (이상치): {outlier_categories.shape[0]}개")

if not outlier_categories.empty:
    print("\n[이상치 카테고리 목록 (1회 등장)]")
    # 이상치 목록이 너무 길 경우 10개만 출력
    print(", ".join(outlier_categories['Category'].tolist()[:10]))
print("-" * 50)

# 5-3. 카테고리 이름 글자수 분포 분석
df_distinct = pd.DataFrame({'Category': distinct_categories})
# 카테고리 이름 길이 계산 (공백 포함)
df_distinct['Length'] = df_distinct['Category'].apply(len)

length_stats = df_distinct['Length'].describe()
median_length = length_stats['50%']
std_dev = length_stats['std']

# 이상적으로 길거나 짧은 카테고리 정의 (중앙값 기준 ± 2*표준편차를 벗어나는 경우)
# 예시 데이터가 작으므로, 임의의 기준으로 설정: 짧음(5 미만), 김(15 초과)
SHORT_THRESHOLD = 5
LONG_THRESHOLD = 15

short_categories = df_distinct[df_distinct['Length'] < SHORT_THRESHOLD]
long_categories = df_distinct[df_distinct['Length'] > LONG_THRESHOLD]

print(f"\n[카테고리 이름 글자수 분포 분석]")
print(f"전체 고유 카테고리 수: {len(distinct_categories)}개")
print(f"카테고리 이름 길이 통계:\n{length_stats.to_string()}")

print(f"\n- 글자수가 {SHORT_THRESHOLD} 미만인 짧은 카테고리: {short_categories.shape[0]}개")
if not short_categories.empty:
    print(f"  목록 ({short_categories.shape[0]}개): {', '.join(short_categories['Category'].tolist())}")

print(f"- 글자수가 {LONG_THRESHOLD} 초과인 긴 카테고리: {long_categories.shape[0]}개")
if not long_categories.empty:
    print(f"  목록 ({long_categories.shape[0]}개): {', '.join(long_categories['Category'].tolist())}")
print("-" * 50)


In [ ]:
books_['category'] = books_['category'].apply(lambda x: str2list(x)[0] if not pd.isna(x) else np.nan)